### Memory Game 

**Projekt-Arbeit Lucas Gamper & Shannon Rüedi**
***
*v1.8*


In [ ]:
# Python Memory Game 
# Authors: Shannon S. & Lucas G.
# Version 1.8

# Import Funktionen
import random, time, os
from ipywidgets import HBox, VBox, Button, Image, Label
from ipycanvas import Canvas

class MemoryGame:
    def __init__(self, images, backside):
        """
        Initialisierung der Klasse MemoryGame
        Deck wird aus der Liste der Bilder erstellt und gemischt.

        Definiert diverse Variablen, und ruft die Funktionen 'update_label' & 'create_board' auf um das Spiel zu starten
        """
        self.images = images * 2
        random.shuffle(self.images)
        self.revealed = [False] * len(self.images)
        self.canvas_board = []
        self.first_card = None
        self.backside = backside 
        self.p1 = p1
        self.p2 = p2
        self.current_player = 1
        self.click_counter = 0
        self.match_counter_p1 = 0  
        self.match_counter_p2 = 0 
        self.match_label_p1 = Label()  
        self.match_label_p2 = Label() 
        self.matches_p1 = HBox([])
        self.matches_p2 = HBox([])
        self.player_label = Label(f"{p1} am Zug")
        self.update_label()
        self.create_board()

    def create_board(self):
        """
        Erstellen des Spielfelds mit verdeckten Karten durch aufrufen der 'draw_back' Funktion
        Callback für Mausklick und aufrufen der 'on_mousedown' Funktion
        """
        self.canvas_board = []
        for idx, images in enumerate(self.images):
            canvas = Canvas(width=150, height=150, layout={"border": "1px solid white"})
            canvas.on_mouse_down(lambda x, y, idx=idx: self.on_mousedown(idx))
            self.draw_back(canvas)
            self.canvas_board.append((canvas, idx))

        self.reset_button = Button(description="Neues Spiel")
        self.reset_button.on_click(self.reset_game)
        
        #UI 'Design' by chatGPT
        self.container = VBox([self.match_label_p1, self.matches_p1, self.match_label_p2, self.matches_p2, self.player_label] + [HBox([canvas for canvas, _ in self.canvas_board[i:i+6]]) for i in range(0, len(self.canvas_board), 6)] + [self.reset_button])

    def on_mousedown(self, idx):
        """
        Spiellogik
        Interpretation vom Input (Mausklick)
        Detailierte Beschreibung in Dokumentation
        """

        canvas, selected_card = self.canvas_board[idx]

        #Karte schon aufgedeckt
        if self.revealed[idx]: 
            return
        
        #1. Karte hat noch keinen Wert -> Ausgewählte Karte wird gezeichnet &in 'first_card' gespeichert
        if self.first_card is None:
            self.first_card = selected_card
            self.draw_card(canvas, idx)
            self.click_counter += 1
        else:

            #Zeichnet 2. Karte 
            self.draw_card(canvas, idx)
            self.click_counter += 1

            #Gleiche Karte wie 'first_card'
            if self.first_card == selected_card:
                return
            
            #Match gefunden
            if self.images[self.first_card] == self.images[selected_card]:
                self.revealed[self.first_card] = True
                self.revealed[selected_card] = True

                matches_display = Image(value=open(self.images[self.first_card], "rb").read(), format='jpg', width=75, height=75)    

                #Match für 'player 1'
                if self.current_player == 1:
                    self.match_counter_p1 += 1
                    self.matches_p1.children += (matches_display,)

                #Match für 'player 2'
                else:
                    self.match_counter_p2 += 1
                    self.matches_p2.children += (matches_display,)

                self.click_counter = 0  
                self.update_label()

                #alle Karten aufgedeckt ?
                revealed_count = sum(self.revealed)
                if revealed_count == len(self.images):  
                    winner = p1 if self.match_counter_p1 > self.match_counter_p2 else p2
                    self.player_label.value = f"Der Gewinner ist {winner}!"

            #Kein Match       
            else:
                revealed_idx = self.first_card
                time.sleep(1)
                self.draw_back(canvas)
                self.draw_back(self.canvas_board[revealed_idx][0])
            self.first_card = None

            #Nach 2. klick -> Spielzugwechsel
            if self.click_counter >= 2:
                self.current_player = 2 if self.current_player == 1 else 1
                self.click_counter = 0
                player = p1 if self.current_player == 1 else p2
                self.player_label.value = f" {player} am Zug"
                
                               
    def draw_card(self, canvas, idx):
        """
        Zeichnet die selektierte Karte auf den Canvas

        Canvas wird gelöscht, image Variable ladet das selektierte Bild mithilfe des Index & Zeichnet dieses auf den Canvas 
        """
        canvas.clear()
        image = Image.from_file(self.images[idx])
        canvas.draw_image(image, 0, 0, width=150, height=150)

    def draw_back(self, canvas):
        """
        Zeichnet die Rückseite der Karte

        Das Bild in der Variable 'backside' wird geladen und auf den Canvas gezeichnet 
        """
        canvas.clear()
        card_back = Image.from_file(self.backside)
        canvas.draw_image(card_back, 0, 0, width=150, height=150)

    def update_label(self):
        """
        Aktualisiert Anzahl gefundener Paare

        Aktualisiert die Labels der Spieler mit der aktuellen Anzahl gefundener Paare
        """
        self.match_label_p1.value = f"Paare {p1}: {self.match_counter_p1}"
        self.match_label_p2.value = f"Paare {p2}: {self.match_counter_p2}"

    def reset_game(self, button=None):
        """
        Funktion für den 'Neues Spiel' Knopf

        Setzt alle Variablen wieder auf den Standardwert, mischt die Karten erneut 
        """
        self.images = images * 2
        self.match_counter_p1 = 0
        self.match_counter_p2 = 0
        self.matches_p1.children = ()
        self.matches_p2.children = ()
        
        self.current_player = 1
        self.click_counter = 0
        self.player_label.value =  f"Spieler {self.p1} am Zug"
        self.update_label()
        random.shuffle(self.images)
        self.revealed = [False] * len(self.images)
        for canvas, _ in self.canvas_board:
            self.draw_back(canvas)

    

# Funktion für Bilder laden
def load_image(image_dir):
    """
    Lädt die Bilder aus dem Pfad 'image_dir' und erstellt daras die Liste 'images'
    """
    for i in range(1, 16):
        file_name = f"bild{i}.jpg"
        full_path = os.path.join(image_dir, file_name)
        images.append(full_path)
    return images


# Variablen setzen 
images = []
image_dir = 'tier_karten'  
backside = ("card_back.jpg")
images = load_image(image_dir)

# Spiel starten
p1 = input("Spieler 1, Wie heisst Du ? ")
p2 = input("Spieler 2, Wie heisst Du ? ")
game = MemoryGame(images, backside)
game.container